In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [2]:
def read_data(source):
    df = pd.read_csv(source)
    
    df = df[:2322]
    
    df['Date'] = pd.to_datetime(df['pay_date'])
    df.index = df['Date']
    del df['Date'] # , df['pay_date'], 
    df.sort_index()
    
    return df

#read dataframe
df = read_data("daily_67_amount.csv")
target = "amount"

In [3]:
# Function: V & Q
def v_q_values(ts):
    v = ts[ts > 0]
    q  = []
    x1 = 1
    for i in range(0, ts.shape[0]):
        if (ts[i] > 0):
            x2 = i+1
            zeros = (ts[x1:x2] == 0).sum(axis = 0)
            if (zeros >= 0):
                q.append(zeros)
                x1 = x2-1
    return v, np.asarray(q)

# Function: Classification
def classification(ts):
    v, q         = v_q_values(ts)
    adi          = sum(q)/len(v)
    cv_squared   = ( sum( ( (v - ts.mean() )**2)/ len(ts) )/ ts.mean() )
    f_type = 'Smooth'
    if (adi > 1.32 and cv_squared < 0.49 ):
      f_type = 'Intermittent'
    elif (adi > 1.32 and cv_squared > 0.49 ):
      f_type = 'Lumpy'
    elif (adi < 1.32 and cv_squared > 0.49 ):
      f_type = 'Erratic'
    print('ADI: ', round(adi, 3), ', CV: ', round(cv_squared, 3), ', Type: ', f_type)
    return adi, cv_squared

# Function: MASE (Mean Absolute Scaled Error)
def mase(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts[i] - ts[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

# Function: RMSE (Root Mean Squared Error)
def rmse(ts, prediction):
    diff = (ts - prediction[:ts.shape[0]])**2
    mse  = diff.mean()
    return mse**(1/2)

############################################################################

# Function: Croston Method ( https://doi.org/10.2307/3007885 )
def croston_method(ts, alpha = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i  = ts.copy(deep = True)
    q_i  = ts.copy(deep = True)
    f_i  = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]  
    q_i[0]        = 1 
    f_i[0]        = v_i[0]/q_i[0]
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = alpha*q[idx_2] + (1 - alpha)*q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = v_i[idx_1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = v_i[idx_1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))    
    return v_i, q_i, f_i

# Function: SBA (Syntetos & Boylan Approximation) Method (  https://doi.org/10.1016/j.ijforecast.2004.10.001 )
def sba_method(ts, alpha = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i = ts.copy(deep = True)
    q_i = ts.copy(deep = True)
    f_i = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]  
    q_i[0]        = 1 
    f_i[0]        = v_i[0]/q_i[0]
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = alpha*q[idx_2] + (1 - alpha)*q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = (1 - alpha/(2))*(v_i[idx_1+1]/q_i[idx_1+1])
            else:
                f_i[idx_1+1] = (1 - alpha/(2))*v_i[idx_1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = (1 - alpha/(2))*v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = (1 - alpha/(2))*v_i[idx_1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))    
    return v_i, q_i, f_i

# Function: SBJ (Shale, Boylan & Johnston) Method ( https://doi.org/10.1057/palgrave.jors.2602031 )
def sbj_method(ts, alpha = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i = ts.copy(deep = True)
    q_i = ts.copy(deep = True)
    f_i = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]  
    q_i[0]        = 1 
    f_i[0]        = v_i[0]/q_i[0]
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = alpha*q[idx_2] + (1 - alpha)*q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = (1 - alpha/(2 - alpha))*(v_i[idx_1+1]/q_i[idx_1+1])
            else:
                f_i[idx_1+1] = (1 - alpha/(2 - alpha))*v_i[idx_1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = (1 - alpha/(2 - alpha))*v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = (1 - alpha/(2 - alpha))*v_i[idx_1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))    
    return v_i, q_i, f_i

# Function: TSB (Teunter, Syntetos & Babai) Method ( https://doi.org/10.1016/j.ejor.2011.05.018 )
def tsb_method(ts, alpha = 0.1, beta = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i = ts.copy(deep = True)
    q_i = ts.copy(deep = True)
    f_i = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]    
    q_i[0]        = 1 
    f_i[0]        = v_i[0]*q_i[0]
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = beta           + (1 - beta)*q_i[idx_1]
            f_i[idx_1+1] = v_i[idx_1+1]*q_i[idx_1+1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = (1 - beta)*q_i[idx_1]
            f_i[idx_1+1] = v_i[idx_1+1]*q_i[idx_1+1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))  
    return v_i, q_i, f_i

# Function: HES (Prestwich et al. 2014) Method ( https://doi.org/10.1016/j.ijforecast.2014.01.006 )
def hes_method(ts, alpha = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i  = ts.copy(deep = True)
    q_i  = ts.copy(deep = True)
    f_i  = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]  
    q_i[0]        = 1 
    f_i[0]        = v_i[0]/q_i[0]
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = alpha*q[idx_2] + (1 - alpha)*q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = v_i[idx_1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 = 0 # v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = v_i[idx_1+1]/(q_i[idx_1+1] + alpha*q[idx_2]/2)
            else:
                f_i[idx_1+1] = v_i[idx_1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))    
    return v_i, q_i, f_i

# Function: LES (Linear-Exponential Smoothing) Method ( https://doi.org/10.1016/j.ijforecast.2020.08.010 )
def les_method(ts, alpha = 0.1, n_steps = 1):
    v, q = v_q_values(ts)
    v_i  = ts.copy(deep = True)
    q_i  = ts.copy(deep = True)
    f_i  = ts.copy(deep = True)
    v_i.values[:] = 0
    q_i.values[:] = 0
    f_i.values[:] = 0
    date_idx      = ts.index  
    v_i[0]        = ts[0]  
    q_i[0]        = 1 
    f_i[0]        = v_i[0]/q_i[0]
    idx_1         = 0
    idx_2         = 0
    for i in range(0, ts.shape[0]-1):
        if (ts[i] > 0):
            idx_1 = ts.index.get_loc(date_idx[i])
            idx_2 =  v.index.get_loc(date_idx[i])
            v_i[idx_1+1] = alpha*v[idx_2] + (1 - alpha)*v_i[idx_1]
            q_i[idx_1+1] = alpha*q[idx_2] + (1 - alpha)*q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = v_i[idx_1+1]/q_i[idx_1+1]
            else:
                f_i[idx_1+1] = v_i[idx_1]
        else:
            idx_1 = ts.index.get_loc(date_idx[i])
            v_i[idx_1+1] = v_i[idx_1]
            q_i[idx_1+1] = q_i[idx_1]
            if (q_i[idx_1+1] != 0):
                f_i[idx_1+1] = (v_i[idx_1+1]/q_i[idx_1+1]) * (1 -  alpha*q[idx_2]/(2*q_i[idx_1+1]))
            else:
                f_i[idx_1+1] = v_i[idx_1]
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))    
    return v_i, q_i, f_i

# Function: SES (Simple Exponential Smoothing) Method ( https://www.industrydocuments.ucsf.edu/tobacco/docs/#id=jzlc0130 )
def ses_method(ts, alpha = 0.1, n_steps = 1):
    f_i      = ts.copy(deep = True)
    date_idx = ts.index 
    f_i.values[:] = 0
    f_i[0]        = ts[0]
    for i in range(0, ts.shape[0]-1):
        idx        = ts.index.get_loc(date_idx[i])
        f_i[idx+1] =  alpha*ts[idx] +  (1 -  alpha)*(f_i[idx])
    idx = pd.date_range(f_i.index[-1], periods = n_steps, freq = '1d')[1:]
    f_i = f_i.append(pd.Series(np.repeat(f_i[-1], len(idx)), index = idx))  
    return f_i

In [4]:
input_data = df.loc[:datetime.strptime("2021-01-04", "%Y-%m-%d")]
ts = input_data[target]
v, q, yhat = croston_method(ts, n_steps = 5, alpha = 0.2)


In [6]:
df[:datetime.strptime("2021-01-05", "%Y-%m-%d")]

,prod_uid,pay_date,amount
Date,,,
2014-11-13,67,2014/11/13,0
2014-11-14,67,2014/11/14,0
2014-11-15,67,2014/11/15,0
2014-11-16,67,2014/11/16,0
2014-11-17,67,2014/11/17,0
...,...,...,...
2021-01-01,67,2021/1/1,0
2021-01-02,67,2021/1/2,1
2021-01-03,67,2021/1/3,0


In [45]:
pred_length = 4
datelist = pd.date_range(datetime.strptime("2021-01-05", "%Y-%m-%d"), datetime.strptime("2021-03-22", "%Y-%m-%d")).tolist()
x = {'begin Date':[],
     'Monday':[],
     'Mon_true':[],
     'Wednesday':[],
     'Wed_true':[],
     'Friday':[],
     'Fri_true':[],
    }

for i in datelist:
    if i.dayofweek == 1:
        input_data = df.loc[:i]
        v, q, yhat = hes_method(input_data["amount"], n_steps = 5, alpha=0.2)
        #yhat = yhat['Forecast']
        predict = yhat[-3:-1].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=3)]
        
        x['begin Date'].append(i)
        x['Monday'].append(int(predict))
        x['Mon_true'].append(sum(true['amount']))
        
    elif i.dayofweek == 3:
        input_data = df.loc[:i]
        v, q, yhat = hes_method(input_data["amount"], n_steps = 5, alpha=0.2)

        predict = yhat[-3:].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=4)]
        
        x['Wednesday'].append(int(predict))
        x['Wed_true'].append(sum(true['amount']))
        
    elif i.dayofweek == 5:
        input_data = df.loc[:i]
        v, q, yhat = hes_method(input_data["amount"], n_steps = 5, alpha=0.2)

        predict = yhat[-2:].sum()
        true = df.loc[i+timedelta(days=3) : i+timedelta(days=4)]
        
        x['Friday'].append(int(predict))
        x['Fri_true'].append(sum(true['amount']))

In [46]:
import math
error = []
mse = 0
result = pd.DataFrame.from_dict(x)

for i in range(len(result)):
    a = result.loc[i, 'Monday'] + result.loc[i, 'Wednesday'] + result.loc[i, 'Friday']
    b = result.loc[i, 'Mon_true'] + result.loc[i, 'Wed_true'] + result.loc[i, 'Fri_true']
    c = (a - b)/a
    mse = mse + pow((a - b), 2)
    error.append(c)

In [47]:
result['error'] = error
print("HES(alpha=0.2) MSE = ", mse/len(result))
result

HES(alpha=0.2) MSE =  70.63636363636364


,begin Date,Monday,Mon_true,Wednesday,Wed_true,Friday,Fri_true,error
0,2021-01-05,2,3,3,0,4,0,0.666667
1,2021-01-12,4,13,6,0,8,0,0.277778
2,2021-01-19,6,1,9,6,4,9,0.157895
3,2021-01-26,4,8,6,13,6,4,-0.562500
4,2021-02-02,4,9,9,4,4,11,-0.411765
5,2021-02-09,6,0,12,0,8,5,0.807692
6,2021-02-16,8,6,12,5,8,12,0.178571
7,2021-02-23,6,3,9,0,4,9,0.368421
8,2021-03-02,2,1,6,3,2,5,0.100000
9,2021-03-09,2,10,3,2,2,1,-0.857143
